In [22]:
import pandas as pd
import folium
from scipy import stats

## Section 1: Data Imports

In [23]:
data = pd.read_csv('combined_data.csv')
data['PRCP'].fillna(0, inplace=True)
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR'] = data['DATE'].dt.year

## Section 2: Calculate the number of PRCP data points for each station and year

In [24]:
station_year_counts = data.groupby(['STATION', 'YEAR']).size().reset_index(name='count')

## Section 3: Filter stations with at least one year with 365 or 366 PRCP data points

In [25]:
valid_stations = station_year_counts.groupby('STATION')['count'].max().reset_index()
valid_stations = valid_stations[valid_stations['count'] >= 365]
filtered_data = data[data['STATION'].isin(valid_stations['STATION'])] #Only keeps the stations that meet the above requirement

## Section 4: Calculate the mean PRCP for each station and multiply by 365

In [26]:
station_avg_prcp = filtered_data.groupby('STATION')['PRCP'].mean() * 365 / 10
valid_stations_avg = station_avg_prcp[station_avg_prcp.notnull()]

## Section 5: Outlier Detection

In [27]:
z_scores = stats.zscore(valid_stations_avg)
threshold = 2
filtered_data = valid_stations_avg[abs(z_scores) <= threshold]

## Section 6: Map Creation

In [28]:
illinois_map = folium.Map(location=[40, -89], zoom_start=6)

## Section 7: Color Function

In [29]:
def get_color(prcp_value):
    scaled_value = (prcp_value - filtered_data.min()) / (filtered_data.max() - filtered_data.min())
    blue = int(255 * (1 - scaled_value))
    red = int(255 * scaled_value)
    return f'#{red:02X}00{blue:02X}'

## Section 8: Adding Markers to Map

In [30]:
for station, prcp_avg in filtered_data.items():
    latitude = data[data['STATION'] == station]['LATITUDE'].values[0]
    longitude = data[data['STATION'] == station]['LONGITUDE'].values[0]
    station_name = station  # Get the station name

    color = get_color(prcp_avg)

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f'Station: {station_name}<br>PRCP (mm) Annual Average: {prcp_avg:.2f}',
    ).add_to(illinois_map)

## Section 9: Legend

In [31]:
legend_html = f'''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; border: 2px solid grey; padding: 5px; text-align: center;">
    <p>PRCP (mm) Annual Average</p>
    <div style="background: linear-gradient(90deg, #0000FF, #800080, #FF0000); height: 20px; width: 200px; margin: auto;"></div>
    <div style="display: flex; justify-content: space-between;">
        <div>{filtered_data.min():.2f}</div>
        <div>{filtered_data.max():.2f}</div>
    </div>
</div>
'''
folium.Marker(
    location=[40, -88],
    icon=folium.DivIcon(html=legend_html)
).add_to(illinois_map)

## Section 10: Display the Map

In [32]:
illinois_map

In [33]:
valid_stations_avg 

STATION
US1ILBN0005    1275.636026
US1ILBN0014     991.096667
US1ILBN0019     962.357809
US1ILBU0008    1060.921477
US1ILCK0012    1052.249586
                  ...     
US1ILMCL007    1074.407138
US1ILMCL013    1162.270234
US1ILMCL017    1742.706783
US1ILMCL023     908.684221
US1ILMCL038    1035.826366
Name: PRCP, Length: 170, dtype: float64